In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM

Using TensorFlow backend.


In [4]:
from commodities import hard,hard_tech,hard_pct

In [59]:
from sklearn.preprocessing import MinMaxScaler
from trading import *
scaler=MinMaxScaler(feature_range=(-1,1))
df=hard_pct.join(hard_tech)['1993-01-02':]

In [60]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In [61]:
data=df.to_numpy()
scaler.fit(data)
data=scaler.transform(data)
models={}
for asset in hard_pct.columns:
    target=df['pct_gold'].to_numpy().reshape(-1,1)
    scaler.fit(target)
    target=scaler.transform(target)
    x,y=multivariate_data(data,target,0,None,32,1,1)
    train_x=x[:5279]
    train_y=y[:5279].reshape(-1,1)
    model=Sequential([
        LSTM(24),
        Dense(1)
    ])
    model.compile(loss='mean_squared_error', optimizer='adam')
    epochs=10
    model.fit(train_x,train_y.reshape(-1,1),batch_size=32,verbose=0,epochs=epochs)
    print('made model for {}'.format(asset))
    models[asset]=model

made model for pct_gold
made model for pct_oil
made model for pct_silver
made model for pct_ruth
made model for pct_irid
made model for pct_pall
made model for pct_rhod
made model for pct_plat


In [101]:
#len(x[5279:]),len(df['pct_plat'].iloc[5279:-32]),len(df[:start])
#models['pct_plat'].predict(x[5279:]),df['pct_plat'][start:]
trading=pd.DataFrame()
for asset,v in models.items():
    predictions=models[asset].predict(x[5279:])
    prices=df[asset].iloc[5279:-32]
    signals=pd.Series([0 for _ in range(len(prices))],index=prices.index)
    for b in range(len(prices)-1):
        if predictions[b]>0:
            signals.iloc[b+1]=1
        else:
            signals.iloc[b+1]=-1
    trading[asset]=eval_signals(prices,signals)
    #print(asset,eval_signals(prices,signals))

In [106]:
def col_map(x):
    return(x[x.find('_')+1:])
trading=trading.rename(columns=col_map)

,gold,oil,silver,ruth,irid,pall,rhod,plat
Date,,,,,,,,
2015-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2015-01-05,0.994762,1.050645,0.991803,1.000000,1.000000,0.997487,1.000000,0.988294
2015-01-06,0.983162,1.094098,0.976607,1.034483,1.000000,0.993737,1.000000,0.983394
2015-01-07,0.975623,1.077908,0.961568,1.034483,1.000000,0.996219,1.020080,0.979350
2015-01-08,0.979491,1.075473,0.959527,1.034483,1.000000,1.002453,1.028442,0.976142
...,...,...,...,...,...,...,...,...
2020-03-05,0.675939,0.542037,0.749167,0.199136,0.346332,0.224059,0.078555,1.114204
2020-03-06,0.661393,0.598248,0.741112,0.199136,0.346332,0.216704,0.072116,1.070710
2020-03-09,0.656210,0.744974,0.752832,0.199136,0.346332,0.224011,0.069930,1.105021


In [107]:
trading.to_csv('lstm_results.csv',index=True)